In [1]:
import numpy as np

In [34]:
import pandas as pd

# Path to the bounding box file
BBOX_DATASET='/home/thejussk/selective-search-for-object-recognition/bbox_eval_sam_grid_4epoch1.csv' 

# Path to the iou evaluation metric file
IOU_DATASET='/home/thejussk/selective-search-for-object-recognition/archive/bboxed/iou_evaluation_sam_pretrained_selective_grid.csv' 

# Loading the bounding box file
final1=pd.read_csv(BBOX_DATASET) 

# Loading the existing iou evaluation metric file
final2=pd.read_csv(IOU_DATASET) 

In [35]:
final1.head(2)

,image_id,list_gt_bboxes,list_pred_bboxes_selective,list_pred_bboxes_sam_grid_4epoch
0,28ff3d4ede34ff71de27c9091b25fc44,"[[0.5771484375, 0.2607421875, 0.7001953125, 0....","[[0.0, 0.9970674486803519, 0.0, 0.997067448680...","[[0.328125, 0.1640625, 0.40625, 0.20703125], [..."
1,8b8046a319a835e4dacf1fb2449d880b,"[[0.4130859375, 0.51953125, 0.8125, 0.74023437...","[[0.12609970674486803, 0.3020527859237537, 0.0...","[[0.65625, 0.578125, 0.84375, 0.75], [0.527343..."


In [36]:
final2.head(2)

,image_id,list_gt_bboxes,list_pred_bboxes_selective,list_pred_bboxes_sam,iou_scores_selective,iou_scores_sam,iou_scores_sum_selective,iou_scores_count_selective,iou_scores_sum_sam,iou_scores_count_sam,iou_scores_sam_pretrained,iou_scores_sum_sam_pretrained,iou_scores_count_sam_pretrained,iou_scores_sam_grid,iou_scores_sum_sam_grid,iou_scores_count_sam_grid
0,28ff3d4ede34ff71de27c9091b25fc44,"[[0.5771484375, 0.2607421875, 0.7001953125, 0....","[[0.0, 0.9970674486803519, 0.0, 0.997067448680...","[[0.08203125, 0.09765625, 0.43359375, 0.527343...","[{'Predicted BBox': [0.0, 0.9970674486803519, ...","[{'Predicted BBox': [0.08203125, 0.09765625, 0...",1.096786,2,1.733756,3,"[{'Predicted BBox': [0.50390625, 0.21875, 0.75...",2.047724,3,"[{'Predicted BBox': [0.52734375, 0.1875, 0.742...",3.883534,6
1,8b8046a319a835e4dacf1fb2449d880b,"[[0.4130859375, 0.51953125, 0.8125, 0.74023437...","[[0.12609970674486803, 0.3020527859237537, 0.0...","[[0.11328125, 0.11328125, 0.5703125, 0.671875]...","[{'Predicted BBox': [0.12609970674486803, 0.30...","[{'Predicted BBox': [0.11328125, 0.11328125, 0...",2.420775,4,4.488439,6,"[{'Predicted BBox': [0.4453125, 0.25390625, 0....",4.002022,6,"[{'Predicted BBox': [0.40625, 0.2890625, 0.843...",1.394117,2


In [37]:
import pandas as pd

def calculate_iou(boxA, boxB):
    """Calculates Intersection over Union (IoU) between two bounding boxes"""

    # Determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    # Compute the area of intersection rectangle (negative area means no overlap)
    inter_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # Compute the area of both boxes 
    boxA_area = (boxA[2] + 1) * (boxA[3] + 1)
    boxB_area = (boxB[2] + 1) * (boxB[3] + 1)

    # Calculate IoU
    iou = inter_area / float(boxA_area + boxB_area - inter_area)

    return iou

def evaluate_image(image_row, pred_column, iou_threshold=0.5):
    """Calculates IoU scores for a single image"""

    pred_boxes = eval(str(image_row[pred_column]))
    gt_boxes = eval(str(image_row['list_gt_bboxes']))
    gt_boxes = [eval(str(i)) for i in gt_boxes]

    results = []
    for pred_box in pred_boxes:
        best_iou = 0
        for gt_box in gt_boxes:
            iou = calculate_iou(pred_box, gt_box)
            best_iou = max(best_iou, iou)  # Get the highest IoU
        # if  best_iou >= iou_threshold:
        results.append({'Predicted BBox':pred_box, 'Best IOU':best_iou, 'Taken?':best_iou >= iou_threshold})

    return results


In [38]:
final1['iou_scores_sam_grid_4epoch']=None

In [39]:
final1

,image_id,list_gt_bboxes,list_pred_bboxes_selective,list_pred_bboxes_sam_grid_4epoch,iou_scores_sam_grid_4epoch
0,28ff3d4ede34ff71de27c9091b25fc44,"[[0.5771484375, 0.2607421875, 0.7001953125, 0....","[[0.0, 0.9970674486803519, 0.0, 0.997067448680...","[[0.328125, 0.1640625, 0.40625, 0.20703125], [...",None
1,8b8046a319a835e4dacf1fb2449d880b,"[[0.4130859375, 0.51953125, 0.8125, 0.74023437...","[[0.12609970674486803, 0.3020527859237537, 0.0...","[[0.65625, 0.578125, 0.84375, 0.75], [0.527343...",None
2,71cfb66b2166800fae0283d2f28b0902,"[[0.0693359375, 0.70703125, 0.1162109375, 0.74...","[[0.6803519061583577, 0.7624633431085044, 0.0,...","[[0.54296875, 0.171875, 0.921875, 0.7578125]]",None
3,f9849624906b15585f9f7f998b34193c,"[[0.3994140625, 0.6162109375, 0.8115234375, 0....","[[0.45454545454545453, 0.5161290322580645, 0.1...","[[0.5, 0.34375, 0.625, 0.453125]]",None
4,8ac52862c96cc0f4a21cb3f5314b2505,"[[0.146484375, 0.537109375, 0.365234375, 0.639...","[[0.16129032258064516, 0.2932551319648094, 0.2...","[[0.09765625, 0.54296875, 0.375, 0.734375], [0...",None
...,...,...,...,...,...
149,ba301b6e6ace2f31edf20efdd78cd286,"[[0.5009765625, 0.2783203125, 0.615234375, 0.3...","[[0.19648093841642228, 0.34310850439882695, 0....","[[0.4765625, 0.2734375, 0.625, 0.40234375]]",None
150,48a92383d627a10126b46b6bfae2bdb5,"[[0.5205078125, 0.24609375, 0.6328125, 0.38769...","[[0.41055718475073316, 0.45454545454545453, 0....","[[0.5078125, 0.234375, 0.6328125, 0.3671875], ...",None
151,8f175ebdd1fb0e26809311ca829077b8,"[[0.3662109375, 0.0654296875, 0.4208984375, 0....","[[0.0, 0.07038123167155426, 0.0381231671554252...","[[0.53125, 0.16796875, 0.640625, 0.296875]]",None
152,fdcdbc6befee8ab9553039895835bad8,"[[0.2783203125, 0.3359375, 0.4599609375, 0.588...","[[0.6275659824046921, 0.7390029325513197, 0.18...","[[0.5234375, 0.25, 0.6171875, 0.375], [0.72265...",None


In [40]:
for index,row in final1.iterrows():
    # Assign the IoU scores to the dataframe
    final1.loc[index,'iou_scores_sam_grid_4epoch']=str(evaluate_image(row, 'list_pred_bboxes_sam_grid_4epoch', iou_threshold=0.5))

In [41]:
final1

,image_id,list_gt_bboxes,list_pred_bboxes_selective,list_pred_bboxes_sam_grid_4epoch,iou_scores_sam_grid_4epoch
0,28ff3d4ede34ff71de27c9091b25fc44,"[[0.5771484375, 0.2607421875, 0.7001953125, 0....","[[0.0, 0.9970674486803519, 0.0, 0.997067448680...","[[0.328125, 0.1640625, 0.40625, 0.20703125], [...","[{'Predicted BBox': [0.328125, 0.1640625, 0.40..."
1,8b8046a319a835e4dacf1fb2449d880b,"[[0.4130859375, 0.51953125, 0.8125, 0.74023437...","[[0.12609970674486803, 0.3020527859237537, 0.0...","[[0.65625, 0.578125, 0.84375, 0.75], [0.527343...","[{'Predicted BBox': [0.65625, 0.578125, 0.8437..."
2,71cfb66b2166800fae0283d2f28b0902,"[[0.0693359375, 0.70703125, 0.1162109375, 0.74...","[[0.6803519061583577, 0.7624633431085044, 0.0,...","[[0.54296875, 0.171875, 0.921875, 0.7578125]]","[{'Predicted BBox': [0.54296875, 0.171875, 0.9..."
3,f9849624906b15585f9f7f998b34193c,"[[0.3994140625, 0.6162109375, 0.8115234375, 0....","[[0.45454545454545453, 0.5161290322580645, 0.1...","[[0.5, 0.34375, 0.625, 0.453125]]","[{'Predicted BBox': [0.5, 0.34375, 0.625, 0.45..."
4,8ac52862c96cc0f4a21cb3f5314b2505,"[[0.146484375, 0.537109375, 0.365234375, 0.639...","[[0.16129032258064516, 0.2932551319648094, 0.2...","[[0.09765625, 0.54296875, 0.375, 0.734375], [0...","[{'Predicted BBox': [0.09765625, 0.54296875, 0..."
...,...,...,...,...,...
149,ba301b6e6ace2f31edf20efdd78cd286,"[[0.5009765625, 0.2783203125, 0.615234375, 0.3...","[[0.19648093841642228, 0.34310850439882695, 0....","[[0.4765625, 0.2734375, 0.625, 0.40234375]]","[{'Predicted BBox': [0.4765625, 0.2734375, 0.6..."
150,48a92383d627a10126b46b6bfae2bdb5,"[[0.5205078125, 0.24609375, 0.6328125, 0.38769...","[[0.41055718475073316, 0.45454545454545453, 0....","[[0.5078125, 0.234375, 0.6328125, 0.3671875], ...","[{'Predicted BBox': [0.5078125, 0.234375, 0.63..."
151,8f175ebdd1fb0e26809311ca829077b8,"[[0.3662109375, 0.0654296875, 0.4208984375, 0....","[[0.0, 0.07038123167155426, 0.0381231671554252...","[[0.53125, 0.16796875, 0.640625, 0.296875]]","[{'Predicted BBox': [0.53125, 0.16796875, 0.64..."
152,fdcdbc6befee8ab9553039895835bad8,"[[0.2783203125, 0.3359375, 0.4599609375, 0.588...","[[0.6275659824046921, 0.7390029325513197, 0.18...","[[0.5234375, 0.25, 0.6171875, 0.375], [0.72265...","[{'Predicted BBox': [0.5234375, 0.25, 0.617187..."


In [42]:
final1['iou_scores_sam_grid_4epoch']=final1['iou_scores_sam_grid_4epoch'].apply(eval)
final1['iou_scores_sum_sam_grid_4epoch']=final1['iou_scores_sam_grid_4epoch'].apply(lambda x:sum([i['Best IOU'] for i in x if i['Taken?']])) # Finding whether the bounding box is taken or not and calculating the sum of the iou
final1['iou_scores_count_sam_grid_4epoch']=final1['iou_scores_sam_grid_4epoch'].apply(lambda x:len([i['Best IOU'] for i in x if i['Taken?']])) # Finding whether the bounding box is taken or not and calculating the count of the iou

In [43]:
final1

,image_id,list_gt_bboxes,list_pred_bboxes_selective,list_pred_bboxes_sam_grid_4epoch,iou_scores_sam_grid_4epoch,iou_scores_sum_sam_grid_4epoch,iou_scores_count_sam_grid_4epoch
0,28ff3d4ede34ff71de27c9091b25fc44,"[[0.5771484375, 0.2607421875, 0.7001953125, 0....","[[0.0, 0.9970674486803519, 0.0, 0.997067448680...","[[0.328125, 0.1640625, 0.40625, 0.20703125], [...","[{'Predicted BBox': [0.328125, 0.1640625, 0.40...",1.057170,2
1,8b8046a319a835e4dacf1fb2449d880b,"[[0.4130859375, 0.51953125, 0.8125, 0.74023437...","[[0.12609970674486803, 0.3020527859237537, 0.0...","[[0.65625, 0.578125, 0.84375, 0.75], [0.527343...","[{'Predicted BBox': [0.65625, 0.578125, 0.8437...",2.457890,3
2,71cfb66b2166800fae0283d2f28b0902,"[[0.0693359375, 0.70703125, 0.1162109375, 0.74...","[[0.6803519061583577, 0.7624633431085044, 0.0,...","[[0.54296875, 0.171875, 0.921875, 0.7578125]]","[{'Predicted BBox': [0.54296875, 0.171875, 0.9...",0.932578,1
3,f9849624906b15585f9f7f998b34193c,"[[0.3994140625, 0.6162109375, 0.8115234375, 0....","[[0.45454545454545453, 0.5161290322580645, 0.1...","[[0.5, 0.34375, 0.625, 0.453125]]","[{'Predicted BBox': [0.5, 0.34375, 0.625, 0.45...",0.985127,1
4,8ac52862c96cc0f4a21cb3f5314b2505,"[[0.146484375, 0.537109375, 0.365234375, 0.639...","[[0.16129032258064516, 0.2932551319648094, 0.2...","[[0.09765625, 0.54296875, 0.375, 0.734375], [0...","[{'Predicted BBox': [0.09765625, 0.54296875, 0...",2.749477,3
...,...,...,...,...,...,...,...
149,ba301b6e6ace2f31edf20efdd78cd286,"[[0.5009765625, 0.2783203125, 0.615234375, 0.3...","[[0.19648093841642228, 0.34310850439882695, 0....","[[0.4765625, 0.2734375, 0.625, 0.40234375]]","[{'Predicted BBox': [0.4765625, 0.2734375, 0.6...",0.979378,1
150,48a92383d627a10126b46b6bfae2bdb5,"[[0.5205078125, 0.24609375, 0.6328125, 0.38769...","[[0.41055718475073316, 0.45454545454545453, 0....","[[0.5078125, 0.234375, 0.6328125, 0.3671875], ...","[{'Predicted BBox': [0.5078125, 0.234375, 0.63...",1.469463,2
151,8f175ebdd1fb0e26809311ca829077b8,"[[0.3662109375, 0.0654296875, 0.4208984375, 0....","[[0.0, 0.07038123167155426, 0.0381231671554252...","[[0.53125, 0.16796875, 0.640625, 0.296875]]","[{'Predicted BBox': [0.53125, 0.16796875, 0.64...",0.945321,1
152,fdcdbc6befee8ab9553039895835bad8,"[[0.2783203125, 0.3359375, 0.4599609375, 0.588...","[[0.6275659824046921, 0.7390029325513197, 0.18...","[[0.5234375, 0.25, 0.6171875, 0.375], [0.72265...","[{'Predicted BBox': [0.5234375, 0.25, 0.617187...",3.235592,4


In [44]:
final3=final2.merge(final1[['image_id','list_pred_bboxes_sam_grid_4epoch','iou_scores_sam_grid_4epoch','iou_scores_sum_sam_grid_4epoch','iou_scores_count_sam_grid_4epoch']],left_on='image_id',right_on='image_id')

In [45]:
final3

,image_id,list_gt_bboxes,list_pred_bboxes_selective,list_pred_bboxes_sam,iou_scores_selective,iou_scores_sam,iou_scores_sum_selective,iou_scores_count_selective,iou_scores_sum_sam,iou_scores_count_sam,iou_scores_sam_pretrained,iou_scores_sum_sam_pretrained,iou_scores_count_sam_pretrained,iou_scores_sam_grid,iou_scores_sum_sam_grid,iou_scores_count_sam_grid,list_pred_bboxes_sam_grid_4epoch,iou_scores_sam_grid_4epoch,iou_scores_sum_sam_grid_4epoch,iou_scores_count_sam_grid_4epoch
0,28ff3d4ede34ff71de27c9091b25fc44,"[[0.5771484375, 0.2607421875, 0.7001953125, 0....","[[0.0, 0.9970674486803519, 0.0, 0.997067448680...","[[0.08203125, 0.09765625, 0.43359375, 0.527343...","[{'Predicted BBox': [0.0, 0.9970674486803519, ...","[{'Predicted BBox': [0.08203125, 0.09765625, 0...",1.096786,2,1.733756,3,"[{'Predicted BBox': [0.50390625, 0.21875, 0.75...",2.047724,3,"[{'Predicted BBox': [0.52734375, 0.1875, 0.742...",3.883534,6,"[[0.328125, 0.1640625, 0.40625, 0.20703125], [...","[{'Predicted BBox': [0.328125, 0.1640625, 0.40...",1.057170,2
1,8b8046a319a835e4dacf1fb2449d880b,"[[0.4130859375, 0.51953125, 0.8125, 0.74023437...","[[0.12609970674486803, 0.3020527859237537, 0.0...","[[0.11328125, 0.11328125, 0.5703125, 0.671875]...","[{'Predicted BBox': [0.12609970674486803, 0.30...","[{'Predicted BBox': [0.11328125, 0.11328125, 0...",2.420775,4,4.488439,6,"[{'Predicted BBox': [0.4453125, 0.25390625, 0....",4.002022,6,"[{'Predicted BBox': [0.40625, 0.2890625, 0.843...",1.394117,2,"[[0.65625, 0.578125, 0.84375, 0.75], [0.527343...","[{'Predicted BBox': [0.65625, 0.578125, 0.8437...",2.457890,3
2,71cfb66b2166800fae0283d2f28b0902,"[[0.0693359375, 0.70703125, 0.1162109375, 0.74...","[[0.6803519061583577, 0.7624633431085044, 0.0,...","[[0.13671875, 0.12109375, 0.41796875, 0.542968...","[{'Predicted BBox': [0.6803519061583577, 0.762...","[{'Predicted BBox': [0.13671875, 0.12109375, 0...",7.713759,12,3.153357,5,"[{'Predicted BBox': [0.125, 0.03515625, 0.9140...",1.710721,3,"[{'Predicted BBox': [0.5625, 0.2265625, 0.8671...",0.962588,1,"[[0.54296875, 0.171875, 0.921875, 0.7578125]]","[{'Predicted BBox': [0.54296875, 0.171875, 0.9...",0.932578,1
3,f9849624906b15585f9f7f998b34193c,"[[0.3994140625, 0.6162109375, 0.8115234375, 0....","[[0.45454545454545453, 0.5161290322580645, 0.1...","[[0.09375, 0.125, 0.38671875, 0.421875], [0.60...","[{'Predicted BBox': [0.45454545454545453, 0.51...","[{'Predicted BBox': [0.09375, 0.125, 0.3867187...",0.590123,1,1.728389,3,"[{'Predicted BBox': [0.11328125, 0.34375, 0.41...",1.615905,2,"[{'Predicted BBox': [0.5078125, 0.59375, 0.796...",3.441251,4,"[[0.5, 0.34375, 0.625, 0.453125]]","[{'Predicted BBox': [0.5, 0.34375, 0.625, 0.45...",0.985127,1
4,8ac52862c96cc0f4a21cb3f5314b2505,"[[0.146484375, 0.537109375, 0.365234375, 0.639...","[[0.16129032258064516, 0.2932551319648094, 0.2...","[[0.09765625, 0.10546875, 0.52734375, 0.527343...","[{'Predicted BBox': [0.16129032258064516, 0.29...","[{'Predicted BBox': [0.09765625, 0.10546875, 0...",3.609196,5,2.670564,4,"[{'Predicted BBox': [0.08203125, 0.3671875, 0....",2.897229,4,"[{'Predicted BBox': [0.48046875, 0.30859375, 0...",4.549731,6,"[[0.09765625, 0.54296875, 0.375, 0.734375], [0...","[{'Predicted BBox': [0.09765625, 0.54296875, 0...",2.749477,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,ba301b6e6ace2f31edf20efdd78cd286,"[[0.5009765625, 0.2783203125, 0.615234375, 0.3...","[[0.19648093841642228, 0.34310850439882695, 0....","[[0.09765625, 0.23828125, 0.35546875, 0.527343...","[{'Predicted BBox': [0.19648093841642228, 0.34...","[{'Predicted BBox': [0.09765625, 0.23828125, 0...",4.396806,7,4.290008,6,"[{'Predicted BBox': [0.11328125, 0.62109375, 0...",2.676452,4,"[{'Predicted BBox': [0.4296875, 0.27734375, 0....",2.842529,4,"[[0.4765625, 0.2734375, 0.625, 0.40234375]]","[{'Predicted BBox': [0.4765625, 0.2734375, 0.6...",0.979378,1
150,48a92383d627a10126b46b6bfae2bdb5,"[[0.5205078125, 0.24609375, 0.6328125, 0.38769...","[[0.41055718475073

In [46]:
final3.to_csv('iou_evaluation_sam_pretrained_selective_grid_4epoch_2.csv',index=False)

In [47]:
df_final=final3[['iou_scores_sum_selective','iou_scores_count_selective',
                 'iou_scores_sum_sam','iou_scores_count_sam',
                 'iou_scores_sum_sam_pretrained','iou_scores_count_sam_pretrained',
                 'iou_scores_sum_sam_grid','iou_scores_count_sam_grid',
                 'iou_scores_sum_sam_grid_4epoch','iou_scores_count_sam_grid_4epoch'
                 ]].sum()

In [48]:
df_final#.iloc[even_indices]#/df_final.iloc[odd_indices]

iou_scores_sum_selective            345.319694
iou_scores_count_selective          550.000000
iou_scores_sum_sam                  431.968251
iou_scores_count_sam                633.000000
iou_scores_sum_sam_pretrained       347.181098
iou_scores_count_sam_pretrained     513.000000
iou_scores_sum_sam_grid             475.840391
iou_scores_count_sam_grid           632.000000
iou_scores_sum_sam_grid_4epoch      444.083837
iou_scores_count_sam_grid_4epoch    562.000000
dtype: float64

In [46]:
345.319694/550

0.6278539890909092

In [47]:
431.968251/633

0.682414298578199

In [48]:
347.181098/513

0.6767662729044834

In [19]:
475.840391/632.000000

0.7529120110759494

In [29]:
508.973861/696.000000

0.731284283045977

In [32]:
485.860413/ 623.000000

0.7798722520064205

In [1]:
444.083837/562

0.7901847633451957